Exemple de prompt pour les données :

Génère 5000 exemples de données d'entraînement pour un modèle NER (Named Entity Recognition) au format spaCy, dans le domaine de la réalisation d'export de notations de tests agroalimentaires
**Labels d'entités à utiliser :**
- NOTATION : numéro ou nom de la notation sur laquelle réaliser un export  (42, 56883, notation efficacité, notation taux d'infestation, etc.)
**Format de sortie requis :**
Liste Python où chaque élément est : [phrase_string, {'entities': [[start, end, label], ...]}]
Les indices start et end correspondent aux positions des caractères dans la phrase.
**Exemples de référence :**
[
    ["Python is one of the easiest languages to learn", 
     {'entities': [[0, 6, 'PROGRAMMING_LANGUAGE']]}],
    
    ["Support vector machines are powerful, but neural networks are more flexible.", 
     {'entities': [[0, 22, 'ALGORITHM_MODEL'], [44, 59, 'ALGORITHM_MODEL']]}],
    
    ["I use Django for web development, and Flask for microservices.", 
     {'entities': [[8, 14, 'FRAMEWORK_LIBRARY'], [41, 46, 'FRAMEWORK_LIBRARY']]}],
    
    ["PostgreSQL is my preferred database for production applications.",
     {'entities': [[0, 10, 'DATABASE']]}],
    
    ["Deploy your React app using Docker and Kubernetes on AWS.",
     {'entities': [[12, 17, 'FRAMEWORK_LIBRARY'], [29, 35, 'TOOL_PLATFORM'], [40, 50, 'TOOL_PLATFORM'], [54, 57, 'TOOL_PLATFORM']]}]
]
**Contraintes importantes :**
1. Crée des phrases variées et naturelles (affirmatives, questions, comparaisons, tutoriels)
2. Varie le nombre d'entités : 20% avec 0 entité, 40% avec 1 entité, 30% avec 2-3 entités, 10% avec 4+ entités
3. Assure-toi que les indices [start, end] sont EXACTEMENT corrects (compte chaque caractère)
4. Pas de chevauchement d'entités
5. Distribution équilibrée entre tous les labels
6. Inclus différents contextes : comparaisons, recommandations, tutoriels, questions, descriptions techniques
7. Quelques phrases sans entités pour améliorer la robustesse du modèle
**Format de sortie :**
Retourne UNIQUEMENT une liste Python valide, sans texte avant ou après, prête à être copiée dans le code.
Commence directement par [ et termine par ]
Génère maintenant les 100 exemples :

Pour le modèle de classification : 


Tu es un assistant spécialisé dans la génération de données d'entraînement pour des modèles de classification NLP.
Je souhaite générer des variations de phrases exprimant l'intention d'exporter une notation dans le cadre d'essais en agroalimentaire. La structure de base est : "Je voudrais exporter la notation <quelquechose>" où <quelquechose> peut être un nom (ex: "alpha", "beta") ou un nombre (ex: "1", "42").
Génère 50 variations diversifiées de cette phrase en suivant ces consignes :
- Varie les formulations (synonymes de "exporter", "notation")
- Varie le niveau de formalité (tutoiement/vouvoiement, registre familier/soutenu)
- Inclus des tournures interrogatives et affirmatives
- Varie la structure syntaxique
- Utilise différents marqueurs temporels ou de contexte
- Mélange les cas avec des noms (texte) et des nombres
- Ajoute quelques variations avec des fautes de frappe mineures ou du langage parlé
Format de sortie : JSON avec la structure suivante
{
  "variations": [
   "la phrase générée"  
]
}

In [1]:
#Installer Spacy (pip install spacy)
#Installer le modèle : python -m spacy download en_core_web_lg (à voir pour un en fr)



In [2]:
#Importer les libs
import spacy
from spacy.training.example import Example
from spacy.util import minibatch
from spacy.lookups import Lookups
from srcdata2 import training_data
import random

/home/lgerard/python/hermes/.venv/lib/python3.12/site-packages/torch/cuda/__init__.py:827: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [49]:
#Nouveaux labels

new_labels = [ 
    "NOTATION"
]

In [50]:
#Charger le modèle

train_data = training_data
nlp = spacy.load("fr_core_news_lg")

In [51]:
#Ajouter ner dans le pipeline
#Cette partie vérifie si le pipeline spaCy contient déjà un composant NER :
#Si non : on ajoute un nouveau composant NER vierge au pipeline
#Si oui : on récupère le composant NER existant (pour ne pas le dupliquer)

if 'ner' not in nlp.pipe_names:
    ner = nlp.add_pipe('ner')
else:
    ner = nlp.get_pipe('ner')


#Cette boucle parcourt vos données d'entraînement pour déclarer tous les types d'entités possibles :
#train_data contient des paires (texte, annotations)
#annotations['entities'] est une liste de tuples comme (début, fin, label)
#ent[2] récupère le label (par exemple "PERSONNE", "LIEU", "ORGANISATION")
#ner.add_label(ent[2]) déclare ce type d'entité au modèle
#Pourquoi c'est nécessaire ? Le modèle doit connaître à l'avance tous les types d'entités qu'il devra prédire.

for data_sample, annotations in train_data:
    for ent in annotations['entities']:
        if ent[2] not in ner.labels:
            ner.add_label(ent[2])

In [52]:
#Disable other pipes (such as classification, POS Tagging, etc)
other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']

In [53]:
#Entraînement

with nlp.disable_pipes(*other_pipes):
    optimizer = nlp.resume_training()
    epochs = 30
    for epoch in range(epochs):
        random.shuffle(train_data) # shuffling the dataset for each epoch 
        losses = {}
        batches = minibatch(train_data, size = 128)
        for batch in batches:
            examples = []
            for text, annotations in  batch:
                doc = nlp.make_doc(text)
                example = Example.from_dict(doc, annotations)
                examples.append(example)
            nlp.update(examples, drop = 0.15, losses = losses)
        print(f'Epoch : {epoch + 1}, Loss : {losses}')

nlp.to_disk('ner_v1.0')

/home/lgerard/python/hermes/.venv/lib/python3.12/site-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "Notation teneur en aldéhydes prête" with entities "[[9, 31, 'NOTATION']]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
/home/lgerard/python/hermes/.venv/lib/python3.12/site-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "La notation taux de rhodotorula doit être sauvegar..." with entities "[[12, 34, 'NOTATION']]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
/home/lgerard/python/hermes/.venv/lib/python3.12/site-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in th

Epoch : 1, Loss : {'ner': np.float32(1676.7653)}
Epoch : 2, Loss : {'ner': np.float32(762.8221)}
Epoch : 3, Loss : {'ner': np.float32(317.15717)}
Epoch : 4, Loss : {'ner': np.float32(290.21246)}
Epoch : 5, Loss : {'ner': np.float32(530.3495)}
Epoch : 6, Loss : {'ner': np.float32(204.95248)}
Epoch : 7, Loss : {'ner': np.float32(122.26197)}
Epoch : 8, Loss : {'ner': np.float32(65.16916)}
Epoch : 9, Loss : {'ner': np.float32(27.848562)}
Epoch : 10, Loss : {'ner': np.float32(16.00448)}
Epoch : 11, Loss : {'ner': np.float32(14.040186)}
Epoch : 12, Loss : {'ner': np.float32(6.087919)}
Epoch : 13, Loss : {'ner': np.float32(3.8693297)}
Epoch : 14, Loss : {'ner': np.float32(1.0984318)}
Epoch : 15, Loss : {'ner': np.float32(2.1458042)}
Epoch : 16, Loss : {'ner': np.float32(0.6461781)}
Epoch : 17, Loss : {'ner': np.float32(1.0800002)}
Epoch : 18, Loss : {'ner': np.float32(0.045307364)}
Epoch : 19, Loss : {'ner': np.float32(0.27837342)}
Epoch : 20, Loss : {'ner': np.float32(0.029137004)}
Epoch : 2

In [59]:
#Vérification

import spacy

nlp_updated = spacy.load("ner_v1.0")
doc = nlp_updated("Je voudrais la notation 34")

print([(ent.text, ent.label_) for ent in doc.ents])

[('34', 'NOTATION')]
